In [64]:
import pandas as pd 

In [65]:
path = ("_data.csv")
df = pd.read_csv(path)
df = df[df["Адрес"].str.contains("Москва", case=False)]
pd.set_option('display.max_columns', None)
df.drop(["Unnamed: 0", "Тип", "Телефоны", "Описание", "Название ЖК", "Серия дома", "Ссылка на объявление"], inplace=True, axis=1)

In [66]:
# Меняем название колонок
rename_map = {'Количество комнат': "number_of_rooms", 'Метро': "metro", 'Адрес': "address", 'Площадь, м2': "apartment_area_m2", 'Дом': "house", 'Парковка': "parking_lot"}
df.rename(columns=rename_map, inplace=True)

In [67]:
# Заполняем пропущенные значения
df["number_of_rooms"].fillna("Апартаменты/свободная планировка", inplace=True)
df["metro"].fillna("не указана", inplace=True)
df["parking_lot"].fillna("нет", inplace=True)

df['Дополнительно'] = df['Дополнительно'].fillna('Нет данных')
df['Высота потолков, м'] = df['Высота потолков, м'].fillna(df['Высота потолков, м'].mean())
df['Лифт'] = df['Лифт'].fillna('Нет данных')
df['Мусоропровод'] = df['Мусоропровод'].fillna('Нет данных')
df['Можно с детьми/животными'] = df['Можно с детьми/животными'].fillna('Нет данных')
df.rename(columns={'Можно с детьми/животными': 'Children/pets allowed','Дополнительно': 'Additionally','Высота потолков, м': 'Ceiling_height',
                   'Лифт': 'Elevator','Мусоропровод': 'Garbage_chute'}, inplace= True)

In [69]:
df.to_csv("data.csv")